# Getting Started with Azure ML Notebooks and Microsoft Sentinel

---

## Pre-requisites
- Log Analytics *Reader* permissions on the Microsoft Sentinel workspace
- Python 3.8 notebook kernel (`Python 3.8 - Azure ML`)

# Contents

1. **Introduction**<br>
   1.1 What is a Jupyter notebook?<br>
   1.2 Running code in notebooks<br><br>
2. **Initializing the notebook and MSTICPy**<br>
3. **Querying Data from Microsoft Sentinel**<br>
   3.1 Verifying Microsoft Sentinel settings<br>
   3.2 (Optional) Configure your Azure Cloud<br>
   3.3 Load a QueryProvider for Microsoft Sentinel<br>
   3.4 Authenticate to the Microsoft Sentinel workspace<br>
   3.5 Test your connection using a MSTICPy built-in Microsoft Sentinel query<br><br>
4. **Configure and test external data providers (VirusTotal and Maxmind GeoLite2)**<br>
   4.1 (Optional) Configure Azure Key Vault to store secrets<br>
   4.2 Testing VirusTotal Lookup<br>
   4.3 Testing IP geolocation lookup with Maxmind GeoLite2<br><br>
5. **Conclusion and Next Steps**<br>
6. **Further Resources**<br>
7. **FAQs - Frequently Asked Questions**



---

# 1. Introduction

This notebook takes you through the basics needed to get started with Azure Machine Learning (ML) Notebooks and Microsoft Sentinel.

It focuses on getting things set up and basic steps to query data.

After you've finished running this notebook you can go on to look at the following notebooks:

- **A Tour of Cybersec notebook features** - this takes you through some of the basic
  features for CyberSec investigation/hunting available to you in notebooks.
- **Configuring your environment** - this covers all of the configuration options for 
  accessing external cybersec resources


Each topic includes 'learn more' sections to provide you with the resource to deep
dive into each of these topics. We encourage you to work through the notebook from start
to finish.

Use these documents for background and more details about the steps in this notebook:
- [Use Jupyter notebooks to hunt for security threats](https://docs.microsoft.com/azure/sentinel/notebooks)
- [Tutorial: Get started with Jupyter notebooks and MSTICPy in Microsoft Sentinel](https://docs.microsoft.com/azure/sentinel/notebook-get-started)

The second document follows the steps in this notebook. Note that this notebook uses a simplified interface for configuring settings rather that the MSTICPy configuration tool described in the documents.


<div style="color: Black; background-color: Khaki; padding: 5px; font-size: 20px">
<p>Please run the the code cells in sequence. Skipping cells will result in errors.</p>
<div style="font-size: 14px">
<p>If you encounter any unexpected errors or warnings please see the FAQ at the end of this notebook.</p>
</div>
</div>


---

## 1.1. What is a Jupyter notebook?

<div style="color: Black; background-color: Gray; padding: 5px; font-size: 15px">
If you're familiar with notebooks, skip this section and go to "2. Initializing
the notebook and MSTICPy" section.
</div>
<br>

You are currently reading a Jupyter notebook. [Jupyter](http://jupyter.org/) is an interactive
development and data manipulation environment presented in a browser.

A Jupyter notebook is a document
made up of cells that contain interactive code, alongside that code's output,
and other items such as text and images (what you are looking at now is a cell of *Markdown* text).

<details>
  <summary>More...</summary>
The name, Jupyter, comes from the core supported programming languages that it supports: **Ju**lia, **Pyt**hon, and **R**.
While you can use any of these languages (and others such as Powershell) we are going to use Python in this notebook.

The majority of the notebooks on the <a href=https://github.com/Azure/Azure-Sentinel-Notebooks>Microsoft Sentinel GitHub repo</a>
are written in Python. Whilst there are pros, and cons to each language, Python is a well-established
language that has a large number of materials and libraries well suited for
data analysis and security investigation, making it ideal for our needs.
</details>

To use a Jupyter notebook you need a Jupyter server that will render the notebook and execute the code within it.
This can take the form of a local [Jupyter installation](https://pypi.org/project/jupyter/),
or a remotely hosted version such as 
[Azure Machine Learning Notebooks](https://docs.microsoft.com/azure/machine-learning/how-to-run-jupyter-notebooks). 

<details>
  <summary>Learn more...</summary>
  <ul>
    <li><a href=https://infosecjupyterbook.com/introduction.html>The Infosec Jupyter Book</a>
    has more details on the technical working of Jupyter.
    </li>
    <li><a href=https://jupyter.org/documentation>The Jupyter Project documentation</a></li>
  </ul>
</details>

<br>



---

## 1.2 Running code in notebooks

The **cell** below is a code cell (note that it looks different from the
cell you are reading). The current cell is known as a *Markdown* cell
and lets you write text (including HTML) and include static images.

Select the code cell (using mouse or cursor keys) below.
Once selected, you can execute the code in it by clicking the "Play" button in the cell, or by pressing Shift+Enter.

<p style="border: solid; padding: 5pt; color: white; background-color: DarkOliveGreen">
<b>Tip</b>: You can identify which cells are code cells by selecting them.<br>
In Azure ML notebooks and VSCode, code cells have a larger border
on the left side with a "Play" button to execute the cell.<br>
In other notebook environments code and markdown cells will have
different styles but it's usually easy to distinguish them.
</p>


In [1]:
# This is our first code cell, it contains basic Python code.
# You can run a code cell by selecting it and clicking
# the Run button (to the left of the cell), or by pressing Shift + Enter.
# Any output from the code will be displayed directly below it.
print("Congratulations, you just ran this code cell")
y = 2 + 2
print("2 + 2 =", y)

Congratulations, you just ran this code cell
2 + 2 = 4


Variables set within a code cell persist between cells meaning you can chain cells together.

In this example we're using the value of `y` from the previous cell.

In [2]:
# Note that output from the last line of a cell is automatically
# sent to the output cell, without needing the print() function.
y + 2

6

Now that you understand the basics we can move onto more complex code.


<details>
  <summary>Learn more about notebooks...</summary>
  <ul>
    <li><a href=https://infosecjupyterbook.com/>The Infosec Jupyter Book</a>
    provides an infosec-specific intro to Python.
    </li>
    <li><a href=https://realpython.com/>Real Python</a>
    is a comprehensive set of Python learnings and tutorials.</li>
  <ul>

</details>


---

# 2. Initializing the notebook and MSTICPy

<details>
  <summary>What are Python packages?</summary>
To avoid having to type (or paste) a lot of complex and repetitive code into
notebook cells, most notebooks rely on third party libraries (known in the Python
world as "packages").

Before you can use a package in your notebook, you need to do two things:

- install the package (although the Azure ML Compute has most common packages pre-installed)
- import the package (or some part of the package - usually a module/file, a function or a class)
</details>

## MSTICPy

**MSTICPy** (pronounced miss-tick-pie) is a Python package of CyberSecurity tools for data retrieval, analysis, enrichment and visualization.

## Initializing notebooks

At the start of most Microsoft Sentinel notebooks you will see an initialization cell like the one below.
This cell is specific to the MSTICPy initialization:

- it defines the minimum versions for Python and MSTICPy needed for this notebook
- it then imports and runs the `init_notebook` function.

<details>
    <summary>More about <i>init_notebook</i>...</summary>
    <p>
`init_notebook` does some of the tedious work of importing other packages, 
checking configuration (we'll get to configuration in a moment) and, optionally,
installing other required packages.</p>
</details>
<br>

<div style="border: solid; padding: 5pt">
<b>Notes: </b>
<p>1. Don't be alarmed if you see configuration warnings (such as "Missing msticpyconfig.yaml").<br>
We haven't configured anything yet, so this is expected.</p>
<p>2. You may also see some warnings about package version conflicts. It is usually safe
to ignore these.</p>
</div>

The `%pip install` line ensures that the latest version of msticpy is installed.

In [ ]:
# import some modules needed in this cell
from IPython.display import display, HTML

display(HTML("Checking upgrade to latest msticpy version"))
%pip install --upgrade --quiet msticpy[azuresentinel]


REQ_PYTHON_VER="3.8"
REQ_MSTICPY_VER="1.5.2"

# initialize msticpy
import msticpy
msticpy.init_notebook(namespace=globals());

---

# 3. Querying Data from Microsoft Sentinel

Once we've done this basic initialization step,
we need to make sure we have configuration to tell MSTICPy how to connect
to your workspace.

This configuration is stored in a configuration file (`msticpyconfig.yaml`).<br>

<details>
  <summary>Learn more...</summary>
  <p>
  Although you don't need to know these details now, you can find more information here:
  </p>
  <ul>
    <li><a href=https://msticpy.readthedocs.io/en/latest/getting_started/msticpyconfig.html >MSTICPy Package Configuration</a></li>
    <li><a href=https://msticpy.readthedocs.io/en/latest/getting_started/SettingsEditor.html >MSTICPy Settings Editor</a></li>
  </ul>
  <p>If you need a more complete walk-through of configuration, we have a separate notebook to help you:</p>
  <ul>
    <li><a href=https://github.com/Azure/Azure-Sentinel-Notebooks/blob/master/ConfiguringNotebookEnvironment.ipynb >Configuring Notebook Environment</a></li>
    <li>And for the ultimate walk-through of how to configure all your `msticpyconfig.yaml` settings
  see the <a href=https://github.com/microsoft/msticpy/blob/master/docs/notebooks/MPSettingsEditor.ipynb >MPSettingsEditor notebook</a></li>
    <li>The Azure-Sentinel-Notebooks GitHub repo also contains an template `msticpyconfig.yaml`, with commented-out sections
  that may also be helpful in finding your way around the settings if you want to dig into things
  by hand.</li>
  </ul>
</details>
<br>

---

## 3.1 Verifying Microsoft Sentinel settings

When you launched this notebook from Microsoft Sentinel a basic configuration file - `config.json` -
was copied to your workspace folder.<br>
You should be able to see this file in the file browser to the left.<br>
This file contains details about your Microsoft Sentinel workspace but has
no configuration settings for other external services that we need.

If you didn't have a `msticpyconfig.yaml` file in your workspace folder the 
`init_notebook` function should have created one for you and populated it
with the Microsoft Sentinel workspace data taken from your config.json.

<p style="border: solid; padding: 5pt; color: white; background-color: DarkOliveGreen"><b>Tip:</b>
If you do not see a "msticpyconfig.yaml" file in your user folder, click the refresh button<br>
at the top of the file browser.
</p>

We can check this now by display the settings.

<details>
    <summary>Multiple Microsoft Sentinel workspaces...</summary>
    <p>If you have multiple Microsoft Sentinel workspaces, you can add
    them in the following configuration cell.</p>
    <p>You can choose to keep one as the default or just delete this entry
    if you always want to name your workspaces explicitly when you 
    connect.
    </p>
</details>

In [12]:
import msticpy
from msticpy.config import MpConfigFile, MpConfigEdit
import os
import pprint

mp_conf = "msticpyconfig.yaml"

# check if MSTICPYCONFIG is already an env variable
mp_env = os.environ.get("MSTICPYCONFIG")
mp_conf = mp_env if mp_env and Path(mp_env).is_file() else mp_conf

if not Path(mp_conf).is_file():
    print(
        "No msticpyconfig.yaml was found!",
        "Please check that there is a config.json file in your workspace folder.",
        "If this is not there, go back to the Microsoft Sentinel portal and launch",
        "this notebook from there.",
        sep="\n"
    )
else:
    mpedit = MpConfigEdit(mp_conf)
    mpconfig = MpConfigFile(mp_conf)
    print("Configured Sentinel workspaces:")
    pprint.pprint(msticpy.settings.settings["AzureSentinel"]["Workspaces"], compact=True)

msticpy.settings.refresh_config()

Label(value='Loading. Please wait.')

Configured Sentinel workspaces:
{'Default': {'ResourceGroup': '{{cookiecutter.resource_group}}',
             'SubscriptionId': '{{cookiecutter.subscription_id}}',
             'TenantId': '{{cookiecutter.tenant_id}}',
             'WorkspaceId': '{{cookiecutter.workspace_id}}'},
 '{{cookiecutter.workspace_name}}': {'ResourceGroup': '{{cookiecutter.resource_group}}',
                                     'SubscriptionId': '{{cookiecutter.subscription_id}}',
                                     'TenantId': '{{cookiecutter.tenant_id}}',
                                     'WorkspaceId': '{{cookiecutter.workspace_id}}'}}


At this stage you should only see two entries in the `Azure Sentinel\Workspaces` section:

- An entry with the name of your Microsoft Sentinel workspace
- An entry named "Default" with the same settings.

## 3.2 (Optional) Configure your Azure Cloud

If you are running in a sovereign or government cloud (i.e. not the Azure global cloud)
you must set up Azure functions to use the correct authentication and
resource management authorities.

<p style="border: solid; padding: 5pt"><b>Note:</b>
This is not required if using the Azure Global cloud (most common)
and you can skip this step.</p>

If the domain of your Microsoft Sentinel or Azure Machine learning does
not end with '.azure.com' you should set the appropriate cloud
for your organization.

If you change to a different cloud, hit **Update** and **Save Settings** to write
the changes to your configuration file.

In [5]:
display(mpedit)
mpedit.set_tab("Azure")


## 3.3 Load a QueryProvider for Microsoft Sentinel

To start, we are going to load up a `QueryProvider`
for Microsoft Sentinel. The `QueryProvider` is the object you use to
querying data from MS Sentinel and make it available to view and analyze in the notebook.
There are two steps to do this:
1. Create the `QueryProvider`
2. run the `connect` function to authenticate to the Sentinel workspace.

<div style="border: solid; padding: 5pt"><b>Note:</b>
If you see a warning "Runtime dependency of PyGObject is missing" when loading the<br>
Microsoft Sentinel driver, please see the FAQ section at the end of this notebook.<br>
The warning does not impact any functionality of the notebooks.
</div>
<br>
<details>
    <summary>More about query providers...</summary>
Query results are always returned as *pandas* DataFrames. If you are new
to using *pandas* look at the **Introduction to Pandas** section at in
the **A Tour of Cybersec notebook features** notebook.
    <p>
    The query provider supports other data sources, as well as Microsoft Sentinel.</p>
<p>
Other data sources supported by the `QueryProvider` class include Microsoft Defender for Endpoint,
Splunk, Microsoft Graph API, Azure Resource Graph but these are not covered here.
    </p>
Most query providers come with a range of built-in queries
for common data operations. You can also a query provider to run custom queries against
Microsoft Sentinel data.

Once you've loaded a QueryProvider you'll normally need to authenticate
to the data source (in this case Microsoft Sentinel).
    <ul>
        <li>
 <a href=https://msticpy.readthedocs.io/en/latest/data_acquisition/DataProviders.html#instantiating-a-query-provider >MSTICPy Documentation</a>.</li>
    </ul>
</details>
<br>


In [6]:
# Refresh any config items that might have been saved
# to the msticpyconfig in the previous steps.
msticpy.settings.refresh_config()

# Initialize a QueryProvider for Microsoft Sentinel
qry_prov = QueryProvider("AzureSentinel")

Please wait. Loading Kqlmagic extension...done


## 3.4 Authenticate to the Microsoft Sentinel workspace

Next we need to authenticate.

The code cell immediately following this section will start the authentication process.

In Azure ML notebooks the authentication will default to using the credentials
you used to authentication to the Azure ML workspace.

More information:

<details>
   <summary>Alternative authentication options</summary>
Instead of using the Azure ML credentials, you can opt to use
one of the following:
<ul>
    <li>Device authentication</li>
    <li>Azure CLI credentials</li>
</ul>

<p><b>Device authentication</b> uses a unique code generated on your client
as an additional authentication factor. When prompted, you copy
the code, open a browser to http://microsoft.com/devicelogin and paste
it in. Then follow the interactive authentication flow.</p>

<b>Azure CLI authentication</b> requires you to logon (in the notebook or 
a terminal) before authenticating to Microsoft Sentinel
<pre>az login</pre>
    
<p>You can change the authentication option used when calling "connect"
with the following.<br>
To force <b>Device authentication</b> add the following parameter
to the connect call
<pre>
qry_prov.connect(ws_config, mp_az_auth=False)
</pre>
</p>
<p>
To use <b>Azure CLI authentication</b>:
<pre>
qry_prov.connect(ws_config, mp_az_auth=["cli"])
</pre>
</p>
</details>


<details>
    <summary>Using WorkspaceConfig</summary>
Loading WorkspaceConfig with no parameters will use the details
of your "Default" workspace (see the Configuring Microsoft Sentinel settings section earlier)<br>

If you want to connect to a specific workspace use this syntax:<br>
   <pre>ws_config = WorkspaceConfig(workspace="WorkspaceName")</pre>
'WorkspaceName' should be one of the workspaces defined in msticpyconfig.yaml
</details>


In [7]:
# Get the default Microsoft Sentinel workspace details from msticpyconfig.yaml

ws_config = WorkspaceConfig()
 
# Connect to Microsoft Sentinel with our QueryProvider and config details
qry_prov.connect(ws_config)

Connecting... Authenticating to Azure.


connected


## 3.5 Test your connection using a MSTICPy built-in Microsoft Sentinel query

To explore queries in more detail see the **A Tour of CyberSec Notebook Features** notebook.


In [8]:
# The time parameters are taken from the qry_prov.query_prov time settings
# attribute, which provides the default query time range. You can
# change interactively this by running qry_prov.query_time.
alerts_df = qry_prov.SecurityAlert.list_alerts(start=qry_prov.query_time.start)

if alerts_df.empty:
    md("The query returned no rows for this time range. You might want to increase the time range")

# display first 5 rows of any results
alerts_df.head() # If you have no data you will just see the column headings displayed

Authenticating to Azure.


TenantId                    TimeGenerated  \
0  8ecf8077-cf51-4820-aadd-14040956f35d 2021-12-21 20:01:15.205000+00:00   
1  8ecf8077-cf51-4820-aadd-14040956f35d 2021-12-21 20:01:15.556000+00:00   
2  8ecf8077-cf51-4820-aadd-14040956f35d 2021-12-21 20:01:15.077000+00:00   
3  8ecf8077-cf51-4820-aadd-14040956f35d 2021-12-21 20:01:16.232000+00:00   
4  8ecf8077-cf51-4820-aadd-14040956f35d 2021-12-21 20:01:15.780000+00:00   

                          AlertDisplayName  \
0      Sign-in from an unfamiliar location   
1             Sign-in from an anonymous IP   
2  Impossible travel to atypical locations   
3              Suspicious inbox forwarding   
4                              Mass delete   

                                 AlertName Severity  \
0      Sign-in from an unfamiliar location      Low   
1             Sign-in from an anonymous IP      Low   
2  Impossible travel to atypical locations      Low   
3              Suspicious inbox forwarding      Low   
4                              Mass delete      Low   

                                                                                           Description  \
0  This indicates suspicious login by JeffL to Amsterdam,Noord-Holland,Netherlands from IP: 117.30....   
1  This indicates suspicious login by JeffL to Amsterdam,Noord-Holland,Netherlands from IP: 117.30....   
2  This indicates suspicious login by JeffL to Amsterdam,Noord-Holland,Netherlands from IP: 117.30....   
3  The user JeffL@seccxpninja.onmicrosoft.com created or updated an inbox forwarding rule that forw...   
4  The user JeffL@seccxpninja.onmicrosoft.com deleted more than 9,448 unique objects in a single se...   

  ProviderName VendorName  \
0          IPC  Microsoft   
1          IPC  Microsoft   
2          IPC  Microsoft   
3         MCAS  Microsoft   
4         MCAS  Microsoft   

                                                   VendorOriginalId  \
0  088efa1064dda0c95843406b3b0326a47854652addd94a236c411a949624776d   
1  a0aa8f7044c258464916a4fb9ae02d9752b374a64bb9ede694e977e22622ba47   
2  4ae272875012e62d309987fbe6994fd9b8c4d0b2b849b93ce0b604f258822ea1   
3  7dff714321c5ef88a13ddbd2674590f98e2c3b0f7599caca792061116a39feb7   
4  519f06a014d7fbe11c449e4e1747153c0e0a8f744a53f7d4d6fe1d9b1385479b   

                          SystemAlertId ResourceId SourceComputerId  \
0  183185f6-0825-6fe3-1f4b-96235a6592f7                               
1  8ca95a45-e423-f834-cbc4-fb1a829a0205                               
2  d875eb38-8d70-a8c1-ca2b-6bdfdb3c57b7                               
3  f220de35-08c6-a363-15e2-af898e933755                               
4  86e1d88c-db02-1664-a6b9-1a2dd8e0ab47                               

                                              AlertType ConfidenceLevel  \
0                                    UnfamiliarLocation                   
1                                        AnonymousLogin                   
2                                      ImpossibleTravel                   
3                    MCAS_ALERT_ANUBIS_INBOX_FORWARDING                   
4  MCAS_ALERT_ANUBIS_DETECTION_REPEATED_ACTIVITY_DELETE                   

   ConfidenceScore  IsIncident                     StartTimeUtc  \
0              NaN       False 2021-12-21 19:00:51.129000+00:00   
1              NaN       False 2021-12-21 19:10:44.195000+00:00   
2              NaN       False 2021-12-21 19:23:24.852000+00:00   
3              NaN       False        2021-12-21 20:02:00+00:00   
4              NaN       False        2021-12-21 20:02:00+00:00   

                        EndTimeUtc                ProcessingEndTime  \
0        2021-12-21 19:58:00+00:00 2021-12-21 20:01:14.896000+00:00   
1        2021-12-21 19:58:00+00:00 2021-12-21 20:01:14.896000+00:00   
2        2021-12-21 19:58:00+00:00 2021-12-21 20:01:14.896000+00:00   
3 2021-12-21 20:49:10.838000+00:00 2021-12-21 20:01:14.896000+00:00   
4 2021-12-21 20:59:08.871000+00:00 2021-12-21 20:01:14.896000+00:00   

  RemediationSteps 

# 4. Configure and test external data providers (VirusTotal and Maxmind GeoLite2)

<div style="border: solid; padding:5pt">
<b>Note: </b>
This section is optional although you are likely to need one or more Threat Intel providers.<br>
You can also choose to use AlienVault OTX, IBM XForce or the MS Sentinel TI table (if
you have configured TI import into MS Sentinel) in place of VirusTotal.<br>
Follow the same procedures for the TI provider(s) of your choice.
</div>

Many Microsoft Sentinel notebooks make use of enrichment services such as Threat Intelligence and IP geo-location. We are going to set up two providers for these in this section.

Since both providers have secret keys associated with their accounts we will also show you how to specify an Azure Key Vault to securely store these settings. This is optional - you can choose to store the keys in your msticpyconfig.yaml.


## 4.1 (Optional) Configure Azure Key Vault to store secrets

To store secrets in Azure Key Vault you need to have access to a Key Vault where you have permissions to read and write secrets.

You can read more about this
<a href=https://msticpy.readthedocs.io/en/latest/getting_started/msticpyconfig.html#specifying-secrets-as-key-vault-secrets >in the MSTICPY docs</a><br>
If you want to skip this step, you can sign up for free accounts with both VirusTotal and MaxMind, until you can take the time to
set up Key Vault storage.
</p>

You will need the following information about the Key Vault:
- Azure Tenant ID (this is usually the same as you Microsoft Sentinel tenant)
- Subscription ID that the KeyVault belongs to
- Vault Name
The ResourceGroup and AzureRegion are needed if you want to create a Key Vault using MSTICPy but are optional if the Vault has already been created.

## Instructions
1. Enter the **TenantId** and **Subscription**
2. Enter the **Vault Name** - note: this is simple name, not the full URI of the Vaul.
3. Click **Update**
4. Click **Save Settings**


In [15]:
display(mpedit)
mpedit.set_tab("Key Vault")


## 4.2 Configure and test Virus Total
We are going to use [VirusTotal](https://www.virustotal.com) (VT) as an example of a popular threat intelligence source.
To use VirusTotal threat intel lookups you will need a VirusTotal account and API key.

You can sign up for a free account at the
[VirusTotal getting started page](https://developers.virustotal.com/v3.0/reference#getting-started) website.

If you are already a VirusTotal user, you can, of course, use your existing key.

<p style="border: solid; padding: 5pt; color: black; background-color: Khaki">
<b>Warning</b> If you are using a VT enterprise key we do not recommend storing this
in the msticpyconfig.yaml file.<br>
MSTICPy supports storage of secrets in
Azure Key Vault if you configured this in the previous step.


As well as VirusTotal, we also support a range
of other threat intelligence providers. You can read more about that here:
[MSTICPy TI Providers](https://msticpy.readthedocs.io/en/latest/data_acquisition/TIProviders.html)

### Instructions

To add the VirusTotal details, run the following cell.

1. Select "VirusTotal" from the **Add prov** drop down
2. Click the **Add** button
3. In the left-side Details panel select **Text** as the Storage option.
4. Paste the API key in the **Value** text box.
5. Click the **Update** button to confirm your changes.

Your changes are not yet saved to your configuration file. To
do this, click on the **Save Settings** button at the bottom of the dialog.

If you are unclear about what anything in the configuration editor means, use the **Help** drop-down. This
has instructions and links to more detailed documentation.


In [10]:
mpe = msticpy.MpConfigEdit()
mpe


Label(value='Loading. Please wait.')

In [11]:
display(mpedit)
mpedit.set_tab("TI Providers")


Our notebooks commonly use IP geo-location information. 
In order to enable this we are going to set up [MaxMind GeoLite2](https://www.maxmind.com)
to provide geolocation lookup services for IP addresses.

GeoLite2 uses a downloaded database which requires an account key to download.
You can sign up for a free account and a license key at 
[The Maxmind signup page - https://www.maxmind.com/en/geolite2/signup](https://www.maxmind.com/en/geolite2/signup).
<br>

<details>
    <summary>Using IPStack as an alernative to GeoLite2...</summary>
    <p>
    For more details see the
    <a href=https://msticpy.readthedocs.io/en/latest/data_acquisition/GeoIPLookups.html >
    MSTICPy GeoIP Providers documentation</a>
    </p>
</details>
<br>

Once, you have an account, run the following cell to add the Maxmind GeopIP Lite details to your configuration.

### Instructions

The procedure is similar to the one we used for VirusTotal:

1. Select the "GeoIPLite" provider from the **Add prov** drop-down
2. Click **Add**
3. Select **Text** Storage and paste the license (API/Auth) key into the text box
4. Click **Update**
5. Click **Save Settings** to write your settings to your configuration.


In [12]:
display(mpedit)
mpedit.set_tab("GeoIP Providers")


---

## 4.1. Testing VirusTotal Lookup

Threat intelligence and IP location are two common enrichments that you might apply to queried data.

Let's test the VirusTotal provider with a known bad IP Address.

<details>
    <summary>Learn more...</summary>
    <p>
    </p>
    <ul>
        <li>More details are shown in the <i>A Tour of Cybersec notebook features</i> notebook</li>
        <li><a href=https://msticpy.readthedocs.io/en/latest/data_acquisition/TIProviders.html >Threat Intel Lookups in MSTICPy</a></li>
    </ul>
</details>
<br>

In [13]:
# Refresh any config items that saved
# to the msticpyconfig in the previous steps.
msticpy.settings.refresh_config()

# Create our TI provider
ti = TILookup()

# Lookup an IP Address
ti_resp = ti.lookup_ioc("85.214.149.236", providers=["VirusTotal"])

ti_df = ti.result_to_df(ti_resp)
ti.browse_results(ti_df, severities="all")

Using Open PageRank. See https://www.domcop.com/openpagerank/what-is-openpagerank


## 4.2 Test IP geolocation lookup with Maxmind GeoLite2

<div style="border: solid; padding: 5pt"><b>Note:</b>
You may see the GeoLite driver downloading its database the first time you run this.
</div>
<br>
<details>
    <summary>Learn more about MSTICPy GeoIP providers...</summary>
    <p>
    <a href=https://msticpy.readthedocs.io/en/latest/data_acquisition/GeoIPLookups.html >MSTICPy GeoIP Providers</a>
    </p>
</details>
<br>


In [14]:
geo_ip = GeoLiteLookup()
raw_res, ip_entity = geo_ip.lookup_ip("85.214.149.236")
display(ip_entity[0])

IpAddress(Address=85.214.149.236, Location={ 'AdditionalData': {},
  'CountryCode': 'DE',
...)

---

# 5. Conclusion and Next Steps

In this notebook, we've gone through the basics of installing MSTICPy and setting up configuration.
We also briefly introduced:

- QueryProviders and querying data from Microsoft Sentinel
- Threat Intelligence lookups using VirusTotal
- Geo-location lookups using MaxMind GeoLite2

## Next Steps
We encourage you to run through the **A Tour of Cybersec notebook features** notebook
to get a better feel for some more of the capabilities of notebooks and MSTICPy.</br>

This notebook includes:

- more examples of queries
- visualizing your data
- brief introduction to using panda to manipulate your data.

Also try out some of the other Microsoft Sentinel notebooks:

- Data Visualization:
    - A Tour of Cybersec notebook features
- Investigation:
    - Guided Triage - Alerts
- Hunting:
    - Entity Explorer - Account
    - Entity Explorer - Windows Host
    - Entity Explorer - Domain and URL
- Simple Machine Learning:
    - Machine Learning in Notebooks Examples

Also check out some of the other sample notebooks in the [Microsoft Sentinel Notebooks GitHub repository](https://github.com/Azure/Azure-Sentinel-Notebooks)


---

# 6. Futher resources

 - [Jupyter Notebooks: An Introduction](https://realpython.com/jupyter-notebook-introduction/)
 - [Threat Hunting in the cloud with Azure Notebooks](https://medium.com/@maarten.goet/threat-hunting-in-the-cloud-with-azure-notebooks-supercharge-your-hunting-skills-using-jupyter-8d69218e7ca0)
 - [MSTICPy documentation](https://msticpy.readthedocs.io/)
 - [Microsoft Sentinel Notebooks documentation](https://docs.microsoft.com/azure/sentinel/notebooks)
 - [The Infosec Jupyterbook](https://infosecjupyterbook.com/introduction.html)
 - [Linux Host Explorer Notebook walkthrough](https://techcommunity.microsoft.com/t5/azure-sentinel/explorer-notebook-series-the-linux-host-explorer/ba-p/1138273)
 - [Why use Jupyter for Security Investigations](https://techcommunity.microsoft.com/t5/azure-sentinel/why-use-jupyter-for-security-investigations/ba-p/475729)
 - [Security Investigtions with Microsoft Sentinel & Notebooks](https://techcommunity.microsoft.com/t5/azure-sentinel/security-investigation-with-azure-sentinel-and-jupyter-notebooks/ba-p/432921)
 - [Pandas Documentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/index.html)
 - [Bokeh Documentation](https://docs.bokeh.org/en/latest/)

---

# 7. FAQs

The following links take you to short articles in the Azure-Sentinel-Notebooks Wiki
that answer common questions.

- [How can I download all Azure-Sentinel-Notebooks notebooks to my Azure ML workspace?](https://github.com/Azure/Azure-Sentinel-Notebooks/wiki/How-can-I-download-all-Azure-Sentinel-Notebooks-notebooks-to-my-Azure-ML-workspace%3F)

- [Can I install MSTICPy by default on a new AML compute?](https://github.com/Azure/Azure-Sentinel-Notebooks/wiki/Can-I-install-MSTICPy-by-default-on-a-new-AML-compute%3F)

- [I see error "Runtime dependency of PyGObject is missing" when I load a query provider](https://github.com/Azure/Azure-Sentinel-Notebooks/wiki/%22Runtime-dependency-of-PyGObject-is-missing%22-error)

- [MSTICPy and other packages do not install properly when switching between the Python 3.6 or 3.8 Kernels](https://github.com/Azure/Azure-Sentinel-Notebooks/wiki/MSTICPy-and-other-packages-do-not-install-properly-when-switching-between-the-Python-3.6-or-3.8-Kernels)

- [My user account/credentials do not get cached between notebook runs - using Azure CLI](https://github.com/Azure/Azure-Sentinel-Notebooks/wiki/Caching-credentials-with-Azure-CLI)

See other FAQs here [Microsoft Sentinel Notebooks wiki](https://github.com/Azure/Azure-Sentinel-Notebooks/wiki/)